# Data Splitting

In this notebook I'm splitting and saving the data for sentiment analysis

## Imports

In [1]:
import gc
import os
from pathlib import Path

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
SEED = 42

## Paths

In [4]:
relative_path = os.path.join("../", "data")

In [5]:
cleaned_reviews_path = os.path.join(relative_path, "2_cleaned_reviews")

In [6]:
sentiment_analysis_data_path = os.path.join(relative_path, "3_sentiment_analysis")
Path(sentiment_analysis_data_path).mkdir(parents=True, exist_ok=True)

## Data

### Loading data

In [7]:
reviews_df = pd.read_parquet(os.path.join(cleaned_reviews_path, "reviews.parquet"))
reviews_df["review"] = reviews_df["review"].str.replace("<p>", " ")

### Dataset overview

In [8]:
reviews_df.sample(n=10, random_state=SEED)

,show_id,user_id,type,datetime,sentiment,subtitle,review,review_score,score
31211,579,26804,movie,2019-02-16 14:28:00,positive,Обзор на фильм 'Аквамен',"Киновселенная DC, которая довольно серьезный к...",-8,10.0
72046,98,65761,movie,2013-02-11 22:56:00,positive,Просторы другой тональности,"Не всякая карта расскажет тебе, чем богато зве...",6,9.0
12824,217,7758,movie,2010-07-19 15:32:00,positive,"«Я была здесь лишь один момент, а потом меня н...",Этот фильм долго пылился на жестком диске и вч...,8,10.0
203242,742,56263,series,2016-09-22 18:20:00,positive,"Между мыслью и реальностью, между движением и ...",Что могу сказать... Никогда не любила сериал п...,-4,NaN
199423,643,33423,series,2010-11-19 15:28:00,positive,"Пацаны смотрят вдаль и верят, но трамвай не е...",О сериале 'Реальные пацаны' узнал еще летом. К...,66,9.0
125237,844,1885,movie,2015-06-04 17:19:00,positive,"Сдох твой Бобик, Джонни!","Нюквисту – «Оскар»! (Тому самому, из шведской ...",9,NaN
165963,495,8245,movie,2010-01-14 02:45:00,positive,И колокола в финале...,Странные я испытал эмоции при просмотре этой л...,10,10.0
61509,263,58981,movie,2020-02-23 21:14:00,positive,"О том, как легко служит злу. и трудно - добру",В сравнение с другими гангстерскими лентами Ск...,7,NaN
132535,63,22230,movie,2012-02-27 17:39:00,positive,Всё есть число? (Амбициозная Miss Danity захот...,"А знаете, этот фильм стоит смотреть одному. Чт...",3,9.0
60474,706,48935,movie,2014-02-23 18:31:00,positive,'Джанго' наоборот.,"Итак, для начала статистика. Данная картина бы...",0,9.0


### Looking at reviews

In [9]:
for review in reviews_df.sample(n=10, random_state=SEED)["review"].values[:3]:
    print(review.replace("<p>", "\n"))
    print("\n")

Киновселенная DC, которая довольно серьезный конкурент Marvel выпустила очередной фильм. Фильм очень динамичный и насыщен экшеном. Есть моменты, которые не особо понятны, например откуда взялась царица 'Атланты', почему ее выкинуло на берег, откуда взялись пираты на подводной лодке. Непонятно, почему многие пишут, что сценарий слабый, все очень логично и понятно - Царь полукровка 'Аквамен', который должен объединить море и сушу, на это он и полукровка, что среднее между двумя мирами! Также замечу, что и реально отличный юмор присутствует, а юмор отлично разбавляет нудное общение, как и бывает во многих фильмах. Спецэффекты очень качественные, но и присутствует природная красота городов на берегу океана, что сразу хочется оказаться там. Конечно хочется, чтобы не все в фильмах было так прекрасно как здесь, но и какой-нибудь негатив оставался! В целом фильм на 9, не без недостатков, но опустим их и округлим число до 10.


Не всякая карта расскажет тебе, чем богато звездное небо, оно может

In the previous step I've removed the scores from the reviews, so it is now safe to continue with baseline model creation.

### Selecting needed columns

In [10]:
class_col = "label"
text_col = "text"

For baseline model we're interested only in `sentiment` and `review_body` columns

In [11]:
df = reviews_df[["sentiment", "review"]]
df = df.rename(columns={"sentiment":class_col, "review":text_col})

In [12]:
del reviews_df
gc.collect()

0

### Splitting the data

In [13]:
train_df, test_dev_df = train_test_split(
    df, test_size=0.15, random_state=SEED, stratify=df[class_col]
)

test_df, dev_df = train_test_split(
    test_dev_df, test_size=0.5, random_state=SEED, stratify=test_dev_df[class_col]
)

train_df.shape, test_df.shape, dev_df.shape

((175556, 2), (15490, 2), (15491, 2))

In [14]:
train_df[class_col].value_counts(normalize=True)

positive    0.720272
neutral     0.150026
negative    0.129702
Name: label, dtype: float64

In [15]:
test_df[class_col].value_counts(normalize=True)

positive    0.720271
neutral     0.150032
negative    0.129697
Name: label, dtype: float64

In [16]:
dev_df[class_col].value_counts(normalize=True)

positive    0.720289
neutral     0.150023
negative    0.129688
Name: label, dtype: float64

## Saving data

In [17]:
LABEL_MAP = {"negative": 0, "neutral": 1, "positive": 2}

## Whole Dataset

In [18]:
train_df["fold"] = "train"
test_df["fold"] = "test"
dev_df["fold"] = "dev"

In [19]:
data = pd.concat([train_df, test_df, dev_df]).reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206537 entries, 0 to 206536
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   label   206537 non-null  category
 1   text    206537 non-null  object  
 2   fold    206537 non-null  object  
dtypes: category(1), object(2)
memory usage: 3.3+ MB


In [20]:
data.to_parquet(os.path.join(sentiment_analysis_data_path, "split_reviews.parquet"))

### Separate splits

In [ ]:
train_df[class_col] = train_df[class_col].map(LABEL_MAP)
dev_df[class_col] = dev_df[class_col].map(LABEL_MAP)
test_df[class_col] = test_df[class_col].map(LABEL_MAP)

In [ ]:
train_df.drop("fold", axis=1).to_parquet(
    os.path.join(sentiment_analysis_data_path, "train.parquet"),
    index=False,
)
dev_df.drop("fold", axis=1).to_parquet(
    os.path.join(sentiment_analysis_data_path, "validation.parquet"),
    index=False,
)
test_df.drop("fold", axis=1).to_parquet(
    os.path.join(sentiment_analysis_data_path, "test.parquet"),
    index=False,
)